# 202: Exampville Mode Choice Logsums

Welcome to Exampville, the best simulated town in this here part of the internet!

Exampville is a demonstration provided with Larch that walks through some of the 
data and tools that a transportation planner might use when building a travel model. 

In [ ]:
import larch, numpy, pandas, os
from larch import P, X

In [ ]:
larch.__version__

In this example notebook, we will walk through the creation of logsums from
an existing tour mode choice model.  First, let's load the data files from
our example.

In [ ]:
hh, pp, tour, skims = larch.example(200, ['hh', 'pp', 'tour', 'skims'])

We'll also load the saved model from the mode choice estimation.

In [ ]:
exampville_mode_choice_file = larch.example(201, output_file='/tmp/exampville_mode_choice.html')
m = larch.read_metadata(exampville_mode_choice_file)

We'll replicate the pre-processing used in the mode choice estimation,
to merge the household and person characteristics into the tours data,
add the index values for the home TAZ's, filter to include only 
work tours, and merge with the level of service skims.  (If this 
pre-processing was computationally expensive, it would probably have
been better to save the results to disk and reload them as needed,
but for this model these commands will run almost instantaneously.)

In [ ]:
raw = tour.merge(hh, on='HHID').merge(pp, on=('HHID', 'PERSONID'))
raw["HOMETAZi"] = raw["HOMETAZ"] - 1
raw["DTAZi"] = raw["DTAZ"] - 1
raw = raw[raw.TOURPURP == 1]
raw = raw.join(
    skims.get_rc_dataframe(
        raw.HOMETAZi, raw.DTAZi,
    )
)

Then we bundle the raw data into the `larch.DataFrames` structure,
as we did for estimation, and attach this structure to the model
as its `dataservice`.

In [ ]:
# For clarity, we can define numbers as names for modes
DA = 1
SR = 2
Walk = 3
Bike = 4
Transit = 5

In [ ]:
dfs = larch.DataFrames(
    co=raw, 
    alt_codes=[DA,SR,Walk,Bike,Transit], 
    alt_names=['DA','SR','Walk','Bike','Transit'],
    ch_name='TOURMODE',
)

m.dataservice = dfs

We'll also initialize a DataFrame to hold the computed logsums.
This data will have one row for each case in our source data,
and a column for each possible destination zone.

In [ ]:
logsums = pandas.DataFrame(
    data=0.0,
    index=raw.index, 
    columns=skims.TAZ_ID
)

The logsums from a Model can be computed using the `Model.logsums` method.
However, if we want the logsums for each possible destination, we'll need
to replace the part of our data that depends on the destination zone, 
writing in the appropriate values for each.  We can simply iterate over the
zones, which is a little bit slow but easy to code.  This isn't a big 
deal, as generating the logsums only needs to be done once after the 
mode choice model is finalized.

In [ ]:
for destination_i, dtaz in enumerate(logsums.columns):
    # Get the LOS data for this destination
    new_data = skims.get_rc_dataframe(
        raw.HOMETAZi, destination_i,
    )
    # Write this data into the model's dataservice.
    dfs.data_co[new_data.columns] = new_data
    # Loading this data runs the pre-processing on
    # the dataservice, to create the arrays needed
    # for computation.
    m.load_data()
    # Lastly, compute the logsums and save them
    # to the new DataFrame.
    logsums[dtaz] = m.logsums()


In [ ]:
logsums.head()

Then we can persist the logsums dataframe to disk, for use in the next
example, where we will estimate a destination choice model.

In [ ]:
logsums.to_pickle('/tmp/logsums.pkl.gz')